In [2]:
import nltk
import os
import pandas as pd
import numpy as np
import json
import re

In [3]:
TMT_MOVIE_PATH = '/Users/susan/Development/cs4300sp2021-cw887-qh75-rz92-yc687-yl698/data/movie_data/movie_data_top3000.csv'

with open(TMT_MOVIE_PATH, 'r') as f:
    movie_df = pd.read_csv(f)

In [4]:
movie_df.columns

Index(['Unnamed: 0', 'rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count',
       'rotten_tomatoes_id'],
      dtype='object')

In [5]:
movie_df[movie_df['movie_title'] == 'The LEGO Movie']

,Unnamed: 0,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,rotten_tomatoes_id
1172,15365,m/the_lego_movie,The LEGO Movie,"Emmet (Chris Pratt), an ordinary LEGO figurine...","Boasting beautiful animation, a charming voice...",PG,"Action & Adventure, Animation, Comedy, Kids & ...","Phil Lord, Christopher Miller, Chris McKay","Dan Hageman, Roy Lee, Christopher Miller, Phil...","Chris Pratt, Elizabeth Banks, Will Arnett, Mor...",...,Certified-Fresh,96.0,250.0,Upright,87.0,222764.0,51,240,11,the_lego_movie


## Get StopWords

In [90]:
all_titles = list(set(movie_df['movie_title']))

In [167]:
# stem titles
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import re


def lemmatize_titles(titles_lst):
    lemmatizer = WordNetLemmatizer()
    lemma_res = []
    stemmer = PorterStemmer()
    stem_res = []
    punc_pattern = r"[-,@\'?\.$%_!\;\"><=]"
    for w in titles_lst:
#         w_sub = re.sub(r"\p{P}(?<![\-.])", " ", w)
#         print(f"w_sub = {w_sub}")
        tokens = w.strip().lower().split()
#         old_tokens = re.findall(r"\b\w\w+\b", w.lower())
#         print(f"old_tokens = {old_tokens}")
        tokens = list(filter(lambda x: len(x) > 1, tokens))
        filtered_tokens = []
        for t in tokens:
            if re.findall(punc_pattern, t):
                t = re.sub('\W+','', t)
                filtered_tokens.append(t)
            else:
                filtered_tokens.append(t)
#         print(f'filtered_tokens = {filtered_tokens}')
        token_lst1 = list(map(lambda x: lemmatizer.lemmatize(x), filtered_tokens))
        token_lst2 = list(map(lambda x: stemmer.stem(x), filtered_tokens))
#         print(f"token_lst1 = {token_lst1}")
#         print(f"token_lst2 = {token_lst2}")
        lemma_res.append(" ".join(token_lst1).strip())
        stem_res.append(" ".join(token_lst2).strip())
    return lemma_res

word_list = ["13:30:12 Yuma! ", "World Wars 2", "WWWs iii ?!", "Ben-Hur"]
print(lemmatize_titles(word_list))

['13:30:12 yuma', 'world war', 'www iii', 'benhur']


In [242]:
def lemmatize_title(title):
    lemmatizer = WordNetLemmatizer()
#     punc_pattern = r"[-,@\'?\.$%_!\;\"><=]"
#     title = re.sub(r"[-,@\'?\.$%_!\;\"><=:]", " ", title)
    title = re.sub('\W+',' ', title)
    tokens = title.strip().lower().split()
    tokens = list(filter(lambda x: len(x) > 1, tokens))
#     print(f"tokens = {tokens}")
    filtered_tokens = []
    for t in tokens:
        filtered_tokens.append(t)
    token_lst1 = list(map(lambda x: lemmatizer.lemmatize(x), filtered_tokens))
    return " ".join(token_lst1).strip()

In [243]:
word_list = ["13:30:12 Yuma! ", "World Wars 2", "WWWs iii ?!", "Ben-Hur", "susan's wolrd"]
print(list(map(lambda x: lemmatize_title(x), word_list)))

['13 30 12 yuma', 'world war', 'www iii', 'ben hur', 'susan wolrd']


In [244]:
all_lemma_titles = lemmatize_titles(all_titles)

In [245]:
bind_titles = list(zip(all_lemma_titles, all_titles))
for info in bind_titles[:20]:
    print(info)

('here come the boom', 'Here Comes the Boom')
('priest', 'Priest')
('halloween h20', 'Halloween H20')
('the mexican', 'The Mexican')
('chicago', 'Chicago')
('the last house on the left', 'The Last House on the Left')
('house of 1000 corps', 'House of 1000 Corpses')
('that thing you do', 'That Thing You Do!')
('hunger', 'Hunger')
('dr dolittle', 'Dr. Dolittle 2')
('transformer', 'Transformers')
('the village', 'The Village')
('secret window', 'Secret Window')
('freddys dead: the final nightmare', "Freddy's Dead: The Final Nightmare")
('my cousin vinny', 'My Cousin Vinny')
('die hard: with vengeance', 'Die Hard: With a Vengeance')
('the devil advocate', "The Devil's Advocate")
('rogue one: star war story', 'Rogue One: A Star Wars Story')
('born on the fourth of july', 'Born on the Fourth of July')
('the motorcycle diary', 'The Motorcycle Diaries')


In [246]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    top_words = nltk.pos_tag([val[0] for val in words_freq])
    
    if not n:
        return top_words
    return top_words[:n]

In [247]:
top_words = get_top_n_words(all_lemma_titles)
print(top_words[:100])
print(f"len(top_words) = {len(top_words)}")

[('the', 'DT'), ('of', 'IN'), ('and', 'CC'), ('in', 'IN'), ('to', 'TO'), ('man', 'NN'), ('day', 'NN'), ('ii', 'JJ'), ('movie', 'NN'), ('for', 'IN'), ('dead', 'JJ'), ('star', 'NN'), ('world', 'NN'), ('you', 'PRP'), ('with', 'IN'), ('on', 'IN'), ('love', 'NN'), ('life', 'NN'), ('last', 'JJ'), ('men', 'NNS'), ('part', 'NN'), ('girl', 'VBD'), ('little', 'JJ'), ('house', 'NN'), ('me', 'PRP'), ('american', 'JJ'), ('it', 'PRP'), ('night', 'NN'), ('king', 'VBG'), ('new', 'JJ'), ('story', 'NN'), ('good', 'JJ'), ('black', 'JJ'), ('my', 'PRP$'), ('war', 'NN'), ('street', 'NN'), ('boy', 'JJ'), ('kid', 'NN'), ('iii', 'JJ'), ('city', 'NN'), ('mr', 'NN'), ('dog', 'NN'), ('time', 'NN'), ('child', 'NN'), ('from', 'IN'), ('friday', 'JJ'), ('christmas', 'NN'), ('big', 'JJ'), ('legend', 'NN'), ('at', 'IN'), ('ghost', 'NN'), ('blue', 'VBP'), ('up', 'RP'), ('trek', 'JJ'), ('secret', 'JJ'), ('final', 'JJ'), ('die', 'NN'), ('dream', 'NN'), ('first', 'RB'), ('return', 'VB'), ('back', 'RB'), ('dark', 'JJ'), ('d

In [252]:
def get_stop_words(tagged_top_words, add_all_till=0):
    stop_types = ['DT', 'IN', 'CC', 'TO', 'PRP$', 'PRP', 'WP']
    stop_words = []
    idx = 0
    while idx < len(tagged_top_words):
        word = tagged_top_words[idx][0]
        word_type = tagged_top_words[idx][1]
        if idx < add_all_till:
            stop_words.append(word)
        else:
            if word_type in stop_types:
                stop_words.append(word)      
        idx += 1
    return stop_words

stop_words = get_stop_words(top_words[:400], 10)
stop_words.append("ii")
stop_words.append("iii")
stop_words.append("iv")
stop_words.append("vi")
print(stop_words)

['the', 'of', 'and', 'in', 'to', 'man', 'day', 'ii', 'movie', 'for', 'you', 'with', 'on', 'me', 'it', 'my', 'from', 'at', 'an', 'all', 'who', 'no', 'we', 'by', 'what', 'bride', 'about', 'like', 'your', 'this', 'after', 'her', 'or', 'into', 'over', 'beyond', 'super', 'under', 'before', 'ii', 'iii', 'iv', 'vi']


In [253]:
idx= 0
for w in top_words:
    if len(w[0]) == 2:
        print(w, idx)
    idx += 1

('of', 'IN') 1
('in', 'IN') 3
('to', 'TO') 4
('ii', 'JJ') 7
('on', 'IN') 15
('me', 'PRP') 24
('it', 'PRP') 26
('my', 'PRP$') 33
('mr', 'NN') 40
('at', 'IN') 49
('up', 'RP') 52
('an', 'DT') 63
('is', 'VBZ') 65
('la', 'VBP') 70
('no', 'DT') 85
('we', 'PRP') 87
('by', 'IN') 91
('go', 'VB') 118
('dr', 'NN') 138
('do', 'VBP') 166
('13', 'CD') 182
('or', 'CC') 184
('iv', 'NN') 219
('be', 'VB') 235
('3d', 'CD') 247
('fu', 'NN') 294
('12', 'CD') 307
('vi', 'JJ') 327
('10', 'CD') 353
('21', 'CD') 358
('oz', 'JJ') 423
('gi', 'VBD') 429
('28', 'CD') 457
('40', 'CD') 478
('ha', 'JJ') 491
('50', 'CD') 527
('am', 'VBP') 601
('60', 'CD') 615
('so', 'RB') 646
('st', 'NN') 703
('ex', 'JJ') 787
('le', 'NN') 867
('el', 'NN') 932
('48', 'CD') 936
('hr', 'NN') 937
('de', 'FW') 961
('30', 'CD') 983
('pi', 'NN') 1003
('he', 'PRP') 1501
('ed', 'JJ') 1534
('je', 'NN') 1546
('39', 'CD') 1610
('33', 'CD') 1759
('d2', 'NN') 1790
('x2', 'NNP') 1838
('tu', 'NN') 1854
('if', 'IN') 1859
('du', 'NN') 1889
('47', 'CD')

In [254]:
# stop_words =  list(filter(lambda x: x not in ['blind', 'angel', 'bride'], stop_words))
# stop_words.append("s")
# stop_words.append("ll")
# print(stop_words)

# Filter Games and Movies with Stop Words

In [255]:
def filter_title(title, stop_words=stop_words):
    filtered_title = lemmatize_title(title).split()
    tokens = list(filter(lambda x: x not in stop_words, filtered_title))
    return " ".join(tokens)

In [256]:
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()
# stemmer.stem('flowers'), lemmatizer.lemmatize('flowers')

In [257]:
for title in all_titles:
    filtered = filter_title(title)
    if ":" in filtered:
        print(title,": ", lemmatize_title(title), ", ", filter_title(title), "!!!!!!!!!!!!!!!!")
    else:
        continue

In [258]:
filtered_title_dict = dict() # <rotten_path, filtered_name>  
inv_filtered_title_dict = dict() # <filtered_name, rotten_path>

count = 0
for idx, row in movie_df.iterrows():
    if type(row['rotten_tomatoes_link']) is str:
        rotten_tmt = row['rotten_tomatoes_link'].split('/')[1]
        filtered_title = filter_title(row['movie_title'], stop_words)
        filtered_title_dict[rotten_tmt] = filtered_title
        if filtered_title not in inv_filtered_title_dict:
            inv_filtered_title_dict[filtered_title] = []
        inv_filtered_title_dict[filtered_title].append(rotten_tmt)

In [259]:
import json

inv_filtered_title = os.path.join('movie_data', 'inv_filtered_title_top3000.json')
filtered_title = os.path.join('movie_data', 'filtered_title_top3000.json')
json.dump(filtered_title_dict, open(filtered_title, 'w+'), indent=4)
json.dump(inv_filtered_title_dict, open(inv_filtered_title, 'w+'), indent=4)

In [260]:
available_games = os.path.join('steamData', 'available_games', 'available_game_filtered_title.csv')
with open(available_games, 'r') as f:
    games_df = pd.read_csv(available_games)

In [261]:
games_df.head(2)

,Unnamed: 0,app_id,name,filtered_name
0,0,945360,Among Us,among u
1,1,730,Counter-Strike: Global Offensive,counter strike global offensive


In [262]:
games_df = games_df[{'name', 'app_id'}]

In [263]:
# movie_data_df['filtered_name'] = movie_data_df.apply(lambda x: get_filtered_mov_name(x), axis=1)

games_df['filtered_name'] = games_df.apply(lambda x: filter_title(x['name'], stop_words), axis=1)

In [264]:
# for idx, row in games_df.iterrows():
#     row['filtered_name'] = filter_title(row['name'], stop_words)

In [265]:
games_df[games_df['app_id'] == 812140]

,app_id,name,filtered_name
192,812140,Assassin's Creed® Odyssey,assassin creed odyssey


In [266]:
games_df.to_csv(available_games)

In [267]:
game_filtered_title_dict = dict() # <app_id, filtered_name>  
game_inv_filtered_title_dict = dict() # <filtered_name, app_id>

count = 0
for idx, row in games_df.iterrows():
    app_id = str(row['app_id'])
    filtered_title = row['filtered_name']
    game_filtered_title_dict[app_id] = filtered_title
    if filtered_title not in game_inv_filtered_title_dict:
        game_inv_filtered_title_dict[filtered_title] = []
    game_inv_filtered_title_dict[filtered_title].append(app_id)

In [268]:
game_filtered_title = os.path.join('steamData', 'available_games', 'inv_filtered_title_lyt.json')
game_inv_filtered_title = os.path.join('steamData', 'available_games', 'filtered_title_lyt.json')
json.dump(game_filtered_title, open(game_filtered_title, 'w+'), indent=4)
json.dump(game_inv_filtered_title, open(game_inv_filtered_title, 'w+'), indent=4)

In [269]:
print(os.path.abspath(game_filtered_title))

/Users/susan/Development/cs4300sp2021-cw887-qh75-rz92-yc687-yl698/data/steamData/available_games/inv_filtered_title_lyt.json


## Get Similarity Score

In [271]:
from collections import Counter

def get_similarity_score(val1, val2):
    val_lst1 = val1.strip().split(' ')
    val_lst2 = val2.strip().split(' ')
    
    val_count1 = dict(Counter(val_lst1))
    val_count2 = dict(Counter(val_lst2))
    
    common_keys = set(val_count1.keys()).intersection(set(val_count2.keys()))
    denom = 0
    for k in common_keys:
        denom += min(val_count1[k], val_count2[k]) 
    return denom*2 / (len(val_lst2) + len(val_lst1))


print(get_similarity_score('lego', 'lego world   '))

0.6666666666666666


In [272]:
games_name_list = list(sorted(list(set(games_df['filtered_name']))))
movies_name_list = list(sorted(list(set(inv_filtered_title_dict.keys()))))

In [273]:
def print_top_k(names, scores, k=10):
    top_k_scores = np.argsort(scores)[::-1][:k]
    for idx in top_k_scores:
        print(names[idx], scores[idx])

In [297]:
games_name_sim_score_list = []
for name in games_name_list:
    games_name_sim_score_list.append(get_similarity_score('assassin creed odyssey', name))

In [298]:
print_top_k(games_name_list, games_name_sim_score_list)

assassin creed odyssey 1.0
assassin creed odyssey fate atlantis 0.75
assassin creed unity 0.6666666666666666
assassin creed brotherhood 0.6666666666666666
assassin creed origin 0.6666666666666666
assassin creed remastered 0.6666666666666666
assassin creed revelation 0.6666666666666666
assassin creed rogue 0.6666666666666666
assassin creed syndicate 0.6666666666666666
assassin creed black flag 0.5714285714285714


In [299]:
def get_top_movie_info(score_lst, games_name_list, k=100, filtered_zeros=True):
    res = []
    sorted_idx = np.argsort(score_lst)[::-1]
    if not k:
        k = len(sorted_idx)
    for i in range(k):
        idx = sorted_idx[i]
        if score_lst[idx] > 0:
            res.append([games_name_list[idx], score_lst[idx]])
    return res

In [300]:
get_top_movie_info(games_name_sim_score_list, games_name_list)

[['assassin creed odyssey', 1.0],
 ['assassin creed odyssey fate atlantis', 0.75],
 ['assassin creed unity', 0.6666666666666666],
 ['assassin creed brotherhood', 0.6666666666666666],
 ['assassin creed origin', 0.6666666666666666],
 ['assassin creed remastered', 0.6666666666666666],
 ['assassin creed revelation', 0.6666666666666666],
 ['assassin creed rogue', 0.6666666666666666],
 ['assassin creed syndicate', 0.6666666666666666],
 ['assassin creed black flag', 0.5714285714285714],
 ['assassin creed deluxe edition', 0.5714285714285714],
 ['assassin creed director cut edition', 0.5],
 ['assassin creed origin curse pharaoh', 0.5],
 ['assassin creed origin hidden one', 0.5],
 ['abyss odyssey', 0.4],
 ['3030 deathwar redux space odyssey', 0.25],
 ['enslaved odyssey west premium edition', 0.25],
 ['witcher assassin king enhanced edition', 0.25]]

In [303]:
import time

def get_sim_scores(games_name_list=games_name_list, movies_name_list=movies_name_list):
    sim_json = dict()

    count = 0
    start = time.time()
    for mov_name in movies_name_list:
        count += 1
        if count % 100 == 0:
            print(f'{count}: {time.time() - start}')
        score_lst = []
        for game_name in games_name_list:
            score_lst.append(get_similarity_score(mov_name, game_name))
        sim_json[mov_name] = get_top_movie_info(score_lst, games_name_list)
    return sim_json

In [304]:
name_sim_dict = get_sim_scores()

100: 2.2241768836975098
200: 3.938603162765503
300: 5.652801990509033
400: 7.356556177139282
500: 9.050330877304077
600: 10.76444697380066
700: 12.566086053848267
800: 14.322952032089233
900: 16.05612802505493
1000: 17.819178104400635
1100: 19.533440113067627
1200: 21.379480123519897
1300: 23.278050184249878
1400: 25.160871982574463
1500: 27.016968965530396
1600: 28.775403022766113
1700: 30.497091054916382
1800: 32.29386305809021
1900: 34.110629081726074
2000: 36.01738119125366
2100: 37.75271201133728
2200: 39.52715015411377
2300: 41.29735207557678
2400: 43.05941295623779
2500: 44.89445614814758
2600: 46.70277714729309
2700: 48.47212815284729


In [321]:
count = 0
for mov_name in movies_name_list:
    if mov_name != "":
        for i in range(len(name_sim_dict[mov_name])):
            if i == 0 and name_sim_dict[mov_name][0][1] >= 0.5:
#                 print(f"=== mov : {mov_name} ==== ")
                count += 1
            if name_sim_dict[mov_name][i][1] >= 0.5:
                pass
#                 print(name_sim_dict[mov_name][i])

print(f"count = {count}")

count = 1033


In [322]:
json.dump(name_sim_dict, open(os.path.join('steamData', '80k_data', 'top3000movie_game_title_similarity.json'),'w+'), indent=4)

In [328]:
movie_inv_filtered_title = os.path.join('movie_data', 'inv_filtered_title_top3000.json')
movie_filtered_title = os.path.join('movie_data', 'filtered_title_top3000.json')
movie_filtered_title_dict = json.load(open(movie_filtered_title, 'r'))
movie_inv_filtered_title_dict = json.load(open(movie_inv_filtered_title, 'r'))

In [329]:
name_sim_dict

{'10 000': [['have 10 second', 0.4],
  ['warhammer 40 000 armageddon', 0.3333333333333333],
  ['warhammer 40 000 regicide', 0.3333333333333333],
  ['warhammer 40 000 mechanicus', 0.3333333333333333],
  ['warhammer 40 000 space marine', 0.2857142857142857],
  ['warhammer 40 000 sanctus reach', 0.2857142857142857],
  ['warhammer 40 000 kill team', 0.2857142857142857],
  ['warhammer 40 000 eternal crusade', 0.2857142857142857],
  ['warhammer 40 000 dawn war', 0.2857142857142857],
  ['warhammer 40 000 inquisitor martyr', 0.2857142857142857],
  ['warhammer 40 000 space wolf', 0.2857142857142857],
  ['plane 10 global 64 bit', 0.2857142857142857],
  ['warhammer 40 000 deathwatch enhanced edition', 0.25],
  ['warhammer 40 000 dawn war soulstorm', 0.25],
  ['warhammer 40 000 dawn war retribution', 0.25],
  ['warhammer 40 000 dawn war dark crusade', 0.2222222222222222],
  ['warhammer 40 000 dawn war game year edition', 0.2]],
 '10 cloverfield lane': [['azur lane crosswave', 0.3333333333333333],


In [330]:
def convert_names_to_id(cur_json, movie_inv_filtered_title_dict=movie_inv_filtered_title_dict, game_inv_filtered_title_dict=game_inv_filtered_title_dict):
    movie_names = movie_inv_filtered_title_dict.keys()
    game_names = game_inv_filtered_title_dict.keys()
    
    res = dict() # <mov_tmt_id, <app_id, score>>
    for mov_filtered_name, rakning_lst in cur_json.items():
        mov_tmt_ids = movie_inv_filtered_title_dict[mov_filtered_name] # list of tmt ids
        
        body = {} # <app_id, score>
        for pair in rakning_lst:
            game_filtered_name = pair[0]
            score = pair[1]
            game_app_ids = game_inv_filtered_title_dict[game_filtered_name] # list of app ids
            for app_id in game_app_ids:
                body[app_id] = score
        
        for tmt_id in mov_tmt_ids:
            res[tmt_id] = body
    return res
    
    

In [331]:
name_sim_dict_2 = convert_names_to_id(name_sim_dict)

In [340]:
name_sim_dict_2['the_lego_movie']

{'332310': 0.6666666666666666,
 '214510': 0.5,
 '249130': 0.5,
 '32440': 0.3333333333333333}

In [336]:
json.dump(name_sim_dict_2, open(os.path.join('steamData', '80k_data', 'top3000movie_game_title_similarity_app_id.json'),'w+'), indent=4)